### Creating packages

Now, let's change gears and look at creating our own packages. If you want to make your code accessible to someone else to use via `pip` install, you need to make it a package. In fact, as you'll see at the end of this section, even if you just want to develop an application, it's much better to be working in a package. I won't show you the internals of setting up a setuptools package, but we'll just go over how you work with it and how it is distributed.

To install a local package, use:

```bash
pip install .
```

This will _copy_ the files into site-packages. If you want to actively develop a module, use this instead (setuptools only, command varies on other tools):

```bash
pip install -e .
```

This uses symlink(s) so that you can edit the local files and immediatly see the changes (after restarting Python, as usual).

If you want to produce an SDist for distributing the source, use


```bash
pip install build
python -m build --sdist
```

If you want to produce a wheel for distributing, use

```bash
python -m build --wheel
```

You'll see old tutorials directly call `python setup.py ...`; if you can possibly avoid doing that, please do! The `setup.py` file is still a good idea for setuptools, but it's not even required there (and doesn't exist for any other packaging software). (It's also quite valid to use pipx to install build, but remember the command is `pyproject-build` if you do that).

## Distributions

### Wheel: fast and simple

A wheel is just a normal zipped file with the extension `.whl`. It contains folders that get copied to specific locations, and a metadata folder.

It _does not_ contain `setup.py`/`setup.cfg`/`pyproject.toml`.


Why use wheels?

* Secure installs - arbitrary code does not run
* Fast installs - files are just copied inplace
* Reliable - does not depend on pretty much anything being on user's machine, including setuptools!
* Faster first imports - pip makes .pyc files when it installs
* Can be tagged for Python version, OS, and/or architecture (supports binaries!).

See <https://pythonwheels.com>

### SDist: Source distribution

This is a `.tar.gz` file holding the files needed to make a wheel. It is often a subset of the files in the GitHub repo, though sometimes it contains generated files, like `version.py` or maybe Cython/SWIG generated source files. If there is no matching wheel (only for projects with binary components, in general), then pip gets the SDist and builds/installs manually.

## Poetry: A breath of fresh air


Let's look at an all-in-one solution: Poetry. It is a bit young, and somewhat opinionated (like all tools replacing a broken standard, it wants to stand out). There are some caveats:

* Should be pure python (no compiled extensions in your code)
* Should be PyPI based (no Conda integration AFAIK)
* Updates to `packaging` take a bit longer to get in (due to PyPA syncing releases with pip and not poetry)
    - Currently can't get wheels on macOS Big Sur, for example, due to a recently fixed packaging issue
    
    
I'm strongly against some of the decisions in Poetry, along with a majority of the other PyPA members and Python core develoeprs. These decisions were mostly made for "application" uses, so they are only problematic when making a library. You can avoid them, just follow the following rules:

* Never add an upper limit to your Python version. `^3.6` should be changed to `>=3.6`. Poetry will _force_ you to add an upper limit if a package you include does this, though, so the bad practice percolates.
* Never add an upper limit to a project you don't heavily depend on unless you know you really don't support some version. It's much more likely that you will support the next version than you won't, and makes a mess for solving this later, and forces you to constantly "maintain" the upper limit.

### Step 1: make a new project

In [ ]:
!poetry new tmp_project

In [ ]:
%ls tmp_project/

In [ ]:
%cat tmp_project/pyproject.toml

The following commands I'll demo in a shell, if I have time.

```bash
# Create a virtual environment, start the poetry.lock file
poetry install

# "Enter" the environment (Ctrl-D or exit to exit)
poetry shell

# Run without entering the environment
poetry run ...

# Add a new package (--dev to make it development only)
# Modifies your pyproject.toml
poetry add rich

# Update the environment and lock files
poetry update

# You can use python -m build, or you can use poetry build
# You can publish to PyPI with poetry publish
# And that's package + environment management!
```

When you publish your package, it makes completely normal wheels, so `pip install` works exactly as expected.

New developers can start developing right away by getting your repository and running `poetry install`. They _even get the dev dependencies_ by default! (which was a brilliant choice, IMO). They start with the lock file if it exists, so they always get what you have, and anyone can run `poetry update` if needed.

## Setuptools: Classic, powerful, verbose

The most powerful (and originally, forced by pip) tool is setuptools. This is a collection of hacks built on top of distutils, which is a collections of hacks to build packages (which was the standard library tool that is now deprecated and may be removed in Python 3.12). There are some awful examples around on using it, so look at <https://scikit-hep.org/developer> for a proper example.

The short version:

* Use declarative `setup.cfg` for everything you can
     - Use file: to read files
     - Always use find: for packages - include or exclude if you need to
     - Always set `python_requires`!
* Logic goes in `setup.py`; often it's just `from setuptools import setup; setup()`
     - Binary extensions go here too
     - You don't need this file at all much of the time.
* Always include a `pyproject.toml`, often it's just 5 or so lines
* Check your `MANIFEST.in` to make sure it's not missing things going into the SDist